# L3: Agentic Sales Pipeline

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Load API tokens for our 3rd party APIs

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'lead_qualification_agents.yaml',
    'lead_tasks': 'lead_qualification_tasks.yaml',
    'email_agents': 'email_engagement_agents.yaml',
    'email_tasks': 'email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [4]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [5]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [6]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [7]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2024-10-24 15:46:22,654 - 140277010979712 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [8]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [9]:
flow.plot()

Graph saved as crewai_flow_graph.html


In [10]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [11]:
emails = await flow.kickoff()

2024-10-24 15:46:22,689 - 140277010979712 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int



# Agent: Lead Data Specialist
## Thought: Thought: I have gathered detailed information about João Moura's professional background and can now proceed to collect company information about Clearbit, including its industry, size, revenue, and market presence.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company profile industry size revenue market presence\"}"
## Tool Output: 

Search results: Title: Clearbit Company Profile 2024: Valuation, Investors, Acquisition
Link: https://pitchbook.com/profiles/company/101066-86
Snippet: Information on acquisition, funding, cap tables, investors, and executives for Clearbit. Use the PitchBook Platform to explore the full profile.
---
Title: Clearbit - Market Share, Competitor Insights in Market Research
Link: https://www.6sense.com/tech/market-research/clearbit-market-share
Snippet: Companies using Clearbit for market-research are majorly from United States with 2,701 customers. 65.46% of Clearbit customers ar



# Agent: Lead Data Specialist
## Thought: Thought: I have encountered blocks while trying to access various sources for company information about Clearbit. I will attempt to find relevant information through another search.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company overview industry size revenue 2023\"}"
## Tool Output: 

Search results: Title: Clearbit Company Profile 2024: Valuation, Investors, Acquisition
Link: https://pitchbook.com/profiles/company/101066-86
Snippet: The current revenue for Clearbit is 00000. How much funding has Clearbit raised over time? Clearbit has raised $17M. Who are Clearbit's investors?
---
Title: Clearbit - Products, Competitors, Financials, Employees ...
Link: https://www.cbinsights.com/company/clearbit
Snippet: Clearbit provides business-to-business (B2B) marketing intelligence solutions. Use the CB Insights Platform to explore Clearbit's full profile.
---
Title: Clearbit - Crunchbase Company Profile & Fu



# Agent: Lead Data Specialist
## Final Answer: 
### Personal Information
- **Name:** João Moura
- **Job Title:** Director of Engineering
- **Role Relevance:** 9/10 (as he leads the engineering team responsible for product development and decision-making.)
- **Professional Background:** João Moura has over 14 years of experience in technology, starting from an early age where he taught web development. He has held various roles, including CTO at Palpiteros, and has been influential in scaling teams and developing AI integrations at Clearbit before its acquisition by HubSpot.

### Company Information
- **Company Name:** Clearbit
- **Industry:** Software & Technical Consulting
- **Company Size:** 121 Employees
- **Revenue:** $41.6 Million (as of 2023)
- **Market Presence:** 8/10 (Clearbit is a significant player in the B2B data activation market and has been recently acquired by HubSpot, enhancing its reach.)

### Summary
João Moura, as the Director of Engineering at Clearbit, plays a c



# Agent: Cultural Fit Analyst
## Thought: Thought: Now that I have gathered Clearbit's cultural values, I need to examine their mission and goals using additional resources to understand better their overall alignment with CrewAI.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://canvasbusinessmodel.com/blogs/mission/clearbit-mission\"}"
## Tool Output: 
Mission, Vision & Core Values of Clearbit – CBM
Skip to content
Framework Templates
Blogs
Search
Cart
0
/
$0.00
 EN
EN
ES
FR
PT
Mission, Vision & Core Values of ClearbitOctober 2, 2024
CLEARBIT BUNDLE
Get Full Bundle:
Porter's Five Forces
$15
$10
BCG Matrix
$15
$10
Canvas
$15
$10
Marketing Mix
$15
$10
PESTLE Analysis
$15
$10
SWOT Analysis
$15
$10
TOTAL: ADD TO CART 
At Clearbit, our mission drives every decision we make, shaping our vision for the future and guiding our core values. We are committed to providing innovative solutions that empower businesses to make data-driven decisions and build strong 



# Agent: Cultural Fit Analyst
## Final Answer: 
### Cultural Alignment Assessment Report

- **Cultural Fit Score**: 9/10
- **Cultural Values Comparison**: Clearbit's values of care, craft, team spirit, truth, initiative, and fun complement CrewAI’s innovation-driven approach and customer-centric focus.
- **Mission Alignment**: Both organizations prioritize data-driven decision-making and fostering innovation to empower companies.
- **Qualitative Comments**: The cultural integration of Clearbit’s operational values with CrewAI's strategic objectives indicates a strong potential for partnership synergy, particularly in driving innovative data solutions for enterprise-level automation. Both companies share a keen interest in enhancing customer experiences through intelligent systems, suggesting a positive trajectory for future collaboration.


# Agent: Lead Scorer and Validator
## Task: Aggregate the collected data and perform the following steps: - Score Calculation: Based on predefine

2024-10-24 15:46:50,546 - 140277010979712 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Scorer and Validator
## Thought: Thought: I have gathered some information on Clearbit's size and market presence, particularly that most companies utilizing Clearbit are small, with 10-50 employees and revenues ranging from $1M to $10M. I will look for more information to help assess the cultural fit and complete the scoring criteria.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company culture values\"}"
## Tool Output: 

Search results: Title: Defining Clearbit's company values
Link: https://clearbit.com/blog/company-values
Snippet: Surfacing values, from the bottom up · being customer centric / empathy for customers · learning · excellence / hard work · self-sufficiency · fun ...
---
Title: Clearbit Mission, Values, Culture & Jobs - Teal
Link: https://www.tealhq.com/company/clearbit
Snippet: Continuous Improvement: Clearbit fosters a culture of continuous improvement, encouraging employees to learn, grow, and innovate to drive th

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [12]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0175


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,116505,111097,5408,38


In [13]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0003


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,1697,1323,374,3


## Inspecting Results

In [14]:
scores = flow.state["score_crews_results"]

In [15]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,João Moura
Job Title,
Role Relevance,0
Professional Background,None
Company Name,CrewAI
Industry,
Company Size,0
Revenue,None
Market Presence,0
Lead Score,85


## Results

In [16]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

CrewAI is ready to help you harness the full potential of AI agent
orchestration. Our platform offers seamless integration for automating processes
across various verticals, driving your efficiency and scalability.    **Take
action today**:   - Schedule a quick demo to see our solution in action: [Book
Your Demo Now](#)   - Explore customized solutions tailored for your business
needs: [Discover Custom Solutions](#)    Don’t miss the chance to elevate your
operational capabilities with CrewAI! Your high lead score indicates you’re
primed for this transformation. Let’s connect and dive deeper into your goals!


## How Complex Can it Get?

In [17]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [18]:
flow = SalesPipeline()
flow.plot()

Graph saved as crewai_flow_graph.html


In [19]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)